In [9]:
# Manage imports
from pathlib import Path
import folium
import AisAudioLabeler as aal
import LabelerUtilities as lu

In [2]:
# Assign data home directory, collection filename, sampling filepath, and clip home directory
data_home = Path("~").expanduser() / "Data" / "AISonobuoy"
collection_filename = "collection-ais.json"
sampling_filepath = Path("GpxAudioLabeler").parent / "data" / "sampling-ais.json"
clip_home = Path("~").expanduser() / "Datasets" / "AISonobuoy"

In [3]:
# Load file describing the collection
collection_path = Path(data_home) / collection_filename
collection = lu.load_json_file(collection_path)

2022-04-27 20:25:13,130 - LabelerUtilities - INFO - Loading /Users/rleclair/Data/AISonobuoy/collection-ais.json


In [4]:
# Load file describing the sampling cases
# NOTE: Currently, the data from this file is unused
sampling = lu.load_json_file(sampling_filepath)

2022-04-27 20:25:13,879 - LabelerUtilities - INFO - Loading data/sampling-ais.json


In [5]:
# For now, assume a single source and hydrophone
if len(collection["sources"]) > 1 or len(collection["hydrophones"]) > 1:
    raise Exception("Only one source and one hydrophone expected")
source = collection["sources"][0]
hydrophone = collection["hydrophones"][0]

In [12]:
hydrophone

{'type': 'bucket',
 'name': 'aisonobuoy-pibuoy-v2',
 'prefix': 'compressed',
 'lat': 28.410858,
 'lon': -80.603861,
 'ele': 0.0}

In [6]:
# Download all source files
if source["name"] != hydrophone["name"] or source["prefix"] != hydrophone["prefix"]:
    raise Exception(
        "Source and hydrophone expected using the same prefix from the same bucket"
    )
aal.download_buoy_objects(
    data_home / source["name"],
    source["name"],
    source["prefix"],
)

2022-04-27 20:25:17,873 - S3Utilities - INFO - Listed objects from bucket aisonobuoy-pibuoy-v2
2022-04-27 20:25:17,875 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1645552824.tar.xz exists
2022-04-27 20:25:17,876 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1645639219.tar.xz exists
2022-04-27 20:25:17,877 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1645725648.tar.xz exists
2022-04-27 20:25:17,878 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1645812009.tar.xz exists
2022-04-27 20:25:17,879 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1645898403.tar.xz exists
2022-04-27 20:25:17,880 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1645984841.tar.xz exists
2022-04-27 20:25:17,882 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1646071221.tar.xz exists
2022-04-27 20:25:17,884 - AisAudioLabeler - INFO - File compressed/ais-pibuoy-v2-mk2-3-1646157610.tar.xz exists
2022-04-2

2022-04-27 20:25:17,982 - AisAudioLabeler - INFO - File compressed/power-pibuoy-v2-mk2-3-1647280838.tar.xz exists
2022-04-27 20:25:17,983 - AisAudioLabeler - INFO - File compressed/power-pibuoy-v2-mk2-3-1647367244.tar.xz exists
2022-04-27 20:25:17,984 - AisAudioLabeler - INFO - File compressed/power-pibuoy-v2-mk2-3-1647453604.tar.xz exists
2022-04-27 20:25:17,986 - AisAudioLabeler - INFO - File compressed/power-pibuoy-v2-mk2-3-1647540002.tar.xz exists
2022-04-27 20:25:17,988 - AisAudioLabeler - INFO - File compressed/power-pibuoy-v2-mk2-3-1647626421.tar.xz exists
2022-04-27 20:25:17,991 - AisAudioLabeler - INFO - File compressed/power-pibuoy-v2-mk2-3-1647864972.tar.xz exists
2022-04-27 20:25:17,992 - AisAudioLabeler - INFO - File compressed/sensors-pibuoy-v2-mk2-3-1645552824.tar.xz exists
2022-04-27 20:25:17,993 - AisAudioLabeler - INFO - File compressed/sensors-pibuoy-v2-mk2-3-1645639219.tar.xz exists
2022-04-27 20:25:17,994 - AisAudioLabeler - INFO - File compressed/sensors-pibuoy-v2

In [7]:
# Load AIS data
ais = aal.write_or_read_ais_pickle(data_home, source)

2022-04-27 20:25:23,996 - AisAudioLabeler - INFO - Reading AIS pickle


In [8]:
# Get hydrophone metadata
hmd = aal.write_or_read_hmd_pickle(data_home, hydrophone)

2022-04-27 20:25:25,129 - AisAudioLabeler - INFO - Reading hydrophone metadata pickle


In [60]:
# Plot the location of every tenth AIS position
# Note: Modulo 10 still works fairly quickly
# See: https://python-visualization.github.io/folium/quickstart.html
m = folium.Map(location=[hydrophone["lat"], hydrophone["lon"]],
              zoom_start=13)
for index, row in ais.iterrows():
    if index % 10000 != 0:
        continue
    folium.Circle(
        radius=0,
        location=[row["lat"], row["lon"]],
        popup=f"{row['mmsi']}\n{row['shiptype']}",
        color="crimson",
        fill=False,
    ).add_to(m)

In [61]:
# Display the map
m